In [5]:
library(dplyr)
library(shiny)
library(rvest)
# library(rPython)
install.packages("rPython")

Installing package into ‘/home/livia/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)
Warning message in install.packages("rPython"):
“installation of package ‘rPython’ had non-zero exit status”

In [ ]:



url <- "https://pre.ufcg.edu.br:8443/ControleAcademicoOnline"

tabelaNotas <- function(matricula, senha) {
    pgsession<-html_session(url)
    pgform<-html_form(pgsession)[[1]]  
    filled_form<-set_values(pgform, login=matricula, senha=senha)
    submit_form(pgsession, filled_form)

    urlLogada <- "https://pre.ufcg.edu.br:8443/ControleAcademicoOnline/Controlador?command=AlunoHistorico"

    page <- jump_to(pgsession, urlLogada)


    table <- page %>% read_html() %>% html_nodes(xpath = "/html/body/div[3]/div[3]/div[3]/table") %>% html_table() 
    df <- as.data.frame(table)
    df_notas <- df %>% select('Disciplina', 'Média', 'Situação')

    df_notas_base <- df_notas %>% filter(Situação != "Em Curso")
    return(df_notas_base)
}

shinyApp(

    ui <- fluidPage( 
        textInput("matricula", "Matrícula",width = 9),
        
        textInput("senha", "Senha",width = 20),
        
        tableOutput('table')

      ),

    server <- function(input, output) {

        output$value <- renderText({
            input$matricula
        })

        output$value <- renderText({
            input$senha
        })
        
        

#         output$table <- renderTable(tabelaNotas(input$matricula, input$senha))
        output$table <- renderTable((predicao(tabelaNotas(input$matricula, input$senha) %>% select('Disciplina', 'Média'))))

    }
)

In [6]:
install.packages("rPython", "/home/user/anaconda3/lib/R/library")

Warning message in install.packages("rPython", "/home/user/anaconda3/lib/R/library"):
“'lib = "/home/user/anaconda3/lib/R/library"' is not writable”

ERROR: Error in install.packages("rPython", "/home/user/anaconda3/lib/R/library"): unable to install packages
